In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os

### Loading the Data from the Data Folder

In [2]:
labels = {'Hello' : 0, 'Thanks': 1}

In [3]:
action_class = []
sequences = []
classes = ['Hello', 'Thanks']
no_sequences = 60
sequence_length = 30

for action in classes:
    for sequence in range(no_sequences):
        window = []
        for frame in range(sequence_length):
            result = np.load(os.path.join('C:\\Users\\Tyron\\Sign_Language_Detection\\SASL_Sign_Language_Detection\\DATA', action, str(sequence), '{}.npy'.format(frame)))
            window.append(result)
        sequences.append(window)
        action_class.append(labels[action])

In [4]:
X = np.array(sequences)
Y = np.array(action_class)

In [5]:
X.shape

(120, 30, 1662)

In [6]:
Y.shape

(120,)

### Creating Training and Testing Sets

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [8]:
Y_train.shape

(108,)

# Creating our LTC model

In [9]:
from ncps.tf import LTC
from ncps import wirings
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
Tensor_Board_2 = tf.keras.callbacks.TensorBoard(log_dir=os.path.join('Logs_2'))

In [11]:
LSTM_Model = tf.keras.Sequential([
    tf.keras.Input(shape=(30, 1662)),
    tf.keras.layers.LSTM(32 ,activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(32 ,activation='relu'),
    tf.keras.layers.Dense(16 ,activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [12]:
LSTM_Model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

In [13]:
LSTM_Model_Hist = LSTM_Model.fit(x=X_train, y=Y_train, epochs=100, callbacks=[Tensor_Board_2])

Epoch 1/100
4/4 [==============================] - 3s 28ms/step - loss: 0.8243 - accuracy: 0.5185
Epoch 2/100
4/4 [==============================] - 0s 26ms/step - loss: 0.7327 - accuracy: 0.5185
Epoch 3/100
4/4 [==============================] - 0s 26ms/step - loss: 0.7104 - accuracy: 0.4537
Epoch 4/100
4/4 [==============================] - 0s 27ms/step - loss: 0.9529 - accuracy: 0.5278
Epoch 5/100
4/4 [==============================] - 0s 27ms/step - loss: 0.6906 - accuracy: 0.5648
Epoch 6/100
4/4 [==============================] - 0s 27ms/step - loss: 0.7031 - accuracy: 0.4815
Epoch 7/100
4/4 [==============================] - 0s 28ms/step - loss: 0.6932 - accuracy: 0.4815
Epoch 8/100
4/4 [==============================] - 0s 34ms/step - loss: 0.6929 - accuracy: 0.5278
Epoch 9/100
4/4 [==============================] - 0s 29ms/step - loss: 0.6915 - accuracy: 0.5185
Epoch 10/100
4/4 [==============================] - 0s 29ms/step - loss: 0.6898 - accuracy: 0.5185
Epoch 11/100
4/4 [=

### Saving the Model

In [15]:
LSTM_Model.save('LSTM_SIGN_LANGUAGE_DETECTION_DEMO.h5')